In [1]:
from dependencies import *
from helper import *
from loaddata import review_data,business_data, rest_data, user_data, elite_udata, non_elite_udata

Using TensorFlow backend.


User Data Loaded
Businesses Data Loaded
Review Data Loaded


In [2]:
cv, bid_2_stars_ne, review_data_ne, bid_2_textenc = give_text_feats_tfidf(review_data)
bid_2_stars_e = review_data.loc[review_data['iselite'] == True].groupby('business_id')['stars'].apply(list).to_dict()

Size of Vocabulary:  15374


In [6]:
xfeat,yfeat= create_dataset_novec(40,rest_data,bid_2_stars_ne,bid_2_stars_e,bid_2_textenc,False)
xtrain, xtest, ytrain, ytest = give_train_test(xfeat, yfeat)

Dimension of Our Dataset:  (4771, 7)


In [3]:
def create_dataset_novec(min_thold, bdata, bid_2_stars, bid_2_stars_e, bid_2_textenc, textenc = False, max_thold=float('inf')):
    basicprint = False
    x1 = []
    x2 = []
    y = []
    bids = bdata['business_id'].values
    for bid in bids:
        if bid in bid_2_stars and bid in bid_2_stars_e and bid in bid_2_textenc:
            stars_e = bid_2_stars_e[bid]
            stars_ne = bid_2_stars[bid]
            lestars = len(stars_e)
            if(lestars >= min_thold and lestars < max_thold):
                x = []
                x.append([Counter(stars_ne)[i] for i in range(1,6)])
                x.append([len(stars_ne)])
                x.append([np.mean(stars_ne)])
                if textenc:
                    if basicprint:
                        print('Num features before text: ', len([item for sublist in x for item in sublist]))
                    x.append(bid_2_textenc[bid])
                    if basicprint:
                        print('Num features after text: ', len([item for sublist in x for item in sublist]))
                        basicprint = False
                x = [item for sublist in x for item in sublist]
                x1.append(x)
                y.append(np.mean(stars_e))                        
    xfeat = np.array(x1)
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    xfeat = imp.fit_transform(xfeat)
    yfeat = np.array(y).reshape(-1,)
    print("Dimension of Our Dataset: ", xfeat.shape)
    return xfeat,yfeat



In [8]:
model = give_reg_model_linear()
test_model(model,xtrain,ytrain,xtest,ytest)
model = give_reg_model_rf(pcadim=7)
test_model(model,xtrain,ytrain,xtest,ytest)
model = give_reg_model_gb()
test_model(model,xtrain,ytrain,xtest,ytest)

Linear
Margin  0.10  0.20  0.30  0.40  0.50
        39.69  71.94  86.18  94.55  98.22
 
Random Forest
Margin  0.10  0.20  0.30  0.40  0.50
        40.63  69.74  85.65  94.55  97.28
 
Gradient Boost
Margin  0.10  0.20  0.30  0.40  0.50
        41.26  71.62  87.43  95.18  97.91
 


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, n_iter_no_change=None, presort='auto',
             random_state=4, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)